# Building and flashing firmware

In order to flash the chip with the right firmware you need to edit the makefile with the good firmware path


## Connect to the chip

In [248]:
import chipwhisperer as cw
scope = cw.scope()

In [254]:
scope

cwlite Device
sn         = 50203220343043543030323238313032
fw_version = 
    major = 0
    minor = 64
    debug = 0
gain = 
    mode = high
    gain = 30
    db   = 24.8359375
adc = 
    state          = False
    basic_mode     = rising_edge
    timeout        = 2
    offset         = 0
    presamples     = 0
    samples        = 5000
    decimate       = 1
    trig_count     = 840110697
    fifo_fill_mode = normal
clock = 
    adc_src       = clkgen_x4
    adc_phase     = 0
    adc_freq      = 29538459
    adc_rate      = 29538459.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1         = serial_rx
    tio2         = serial_tx
    tio3         = high_z
    tio4         = high_z
    pdid         = high_z
    pdic         = high_z


In [255]:
target = cw.target(scope, cw.targets.SimpleSerial) #cw.targets.SimpleSerial can be omitted

In [256]:
target

SimpleSerial Settings =
	output_len             = 16
	baud                   = 38400
	simpleserial_last_read = 
	simpleserial_last_sent = 
	

In [257]:
scope.default_setup()

In [284]:
path_to_src = "/home/etienne/Documents/Ecole/ENSIBS/PEI-2/ProjetPluri/Power_Analysis_On_RSA_With_ChipWhisperer/src/"
file_name = "simpleserial-homemade-rsa-CW308_STM32F3.hex"
xor_path = "/home/etienne/Documents/Ecole/ENSIBS/PEI-2/ProjetPluri/nos_trucs/xor_test/simpleserial-xor-CW303.hex"
#cw.program_target(scope, cw.programmers.STM32FProgrammer, path_to_src + file_name)
cw.program_target(scope, cw.programmers.XMEGAProgrammer, xor_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2231 bytes


In [307]:
msg = bytearray([1]*16) #simpleserial uses bytearrays
#target.simpleserial_write('k', msg)

target.simpleserial_write('z', msg)

In [308]:
#print(target.simpleserial_read('r', 16))
print(target.read())
#"r11111111"

`````````````````````````h```h```h```


In [305]:
cle = 1234 #simpleserial uses bytearrays
from struct import pack
target.simpleserial_write('s', pack("<Q", cle))

In [306]:
print(target.simpleserial_read('k', 20))

(ChipWhisperer Target WARNING|File SimpleSerial.py:410) Unexpected start to command: `


None


## Communication with the Target

Communication with targets, which is done through the `SimpleSerial target` object we got earlier, is grouped into two categories:

1. Raw serial via `target.read()`, `target.write()`, `target.flush()`, etc. 

1. SimpleSerial commands via `target.simpleserial_read()`, `target.simpleserial_write()`, `target.simpleserial_wait_ack()`, etc.

The firmware we uploaded uses the simpleserial protocol (https://wiki.newae.com/SimpleSerial), so we'll start off with simpleserial. Later, we'll use the raw serial commands to send the same messages.

If you check the simpleserial-base firmware (`simpleserial-base.c`) you'll find that for the simpleserial `'p'` command, the target will echo back the command.

Now test if your hadware respond to your commands

## Capturing traces 

For capture trace let's arm the target : 

1. Arm the ChipWhisperer with `scope.arm()`. It will begin capturing as soon as it is triggered (which in our case is a rising edge on `gpio4`.
1. `scope.capture()` will read back the captured power trace, blocking until either ChipWhisperer is done recording, or the scope times out. Note that the error return will tell you whether or not the scope timed out. It does not return the captured scope data.
1. You can read back the captured power trace with `scope.get_last_trace()`.

`simpleserial_base` will trigger the ChipWhisperer when we send the `'p'` command. Try capturing a trace now:

ChipWhisperer also has a `capture_trace()` convience function that:

1. Optionally sends the `'k'` command
1. Arms the scope
1. Sends the `'p'` command
1. Captures the trace
1. Reads the return `'r'` message
1. Returns a `Trace` class that groups the trace data, `'p'` message, the `'r'` message, and the `'k'` message

It isn't always the best option to use, but it's usually sufficient for most simpleserial applications

## Troubleshooting
If you get the following error :

```console
.
Compiling:
    simpleserial-homemade-rsa.c ...In file included from utils/utils.h:4,
                 from simpleserial-homemade-rsa.c:3:
/usr/lib/gcc/arm-none-eabi/13.2.0/include/stdint.h:9:16: fatal error: stdint.h: No such file or directory
    9 | # include_next <stdint.h>
      |                ^~~~~~~~~~
compilation terminated.
```


You need to install the following package: `arm-none-eabi-newlib`